In [2]:
import pandas as pd
import urllib.parse
import requests
import pickle
import os
import re
import time

# %load_ext nb_black

# https://github.com/sshh12/csgo-market-analysis/blob/main/Analysis.ipynb

In [3]:
color_to_rare = {
    "b0c3d9": "Consumer Grade",
    "5e98d9": "Industrial Grade",
    "4b69ff": "Mil-Spec",
    "8847ff": "Restricted",
    "d32ce6": "Classified",
    "eb4b4b": "Covert",
    "ffd700": "Special",
}


def parse_weapon(weap_json):
    name = weap_json["name"]
    stattrak = "StatTrak" in name
    name_match = re.match(r"([★™\w\- ]+) \| ([\w ]+) \(([\w\- ]+)\)", name)
    is_case = name.endswith(" Case")
    is_package = name.endswith(" Package")
    is_key = name.endswith(" Case Key")
    if name_match is None and not is_case and not is_key and not is_package:
        return None
    if not is_case and not is_key and not is_package:
        weapon = name_match.group(1).replace("★ ", "").replace("StatTrak™ ", "")
        skin = name_match.group(2)
        cond = name_match.group(3)
    elif is_key:
        skin = "n/a"
        cond = "n/a"
        weapon = "Key"
    elif is_package:
        skin = "n/a"
        cond = "n/a"
        weapon = "Package"
    elif is_case:
        skin = "n/a"
        cond = "n/a"
        weapon = "Case"
    if weapon in ["Sticker", "Sealed Graffiti"]:
        return None
    url = "https://steamcommunity.com/market/listings/730/" + urllib.parse.quote(name)
    data = {
        "name": name,
        "stattrak": stattrak,
        "weapon": weapon,
        "skin": skin,
        "condition": cond,
        "sell_price": weap_json["sell_price"] / 100,
        "sell_listings": weap_json["sell_listings"],
        "url": url,
        "is_key": is_key,
        "is_case": is_case,
        "is_package": is_package,
    }
    return data


def get_prices(resp):
    df_data = []
    for match in re.finditer(r'\["([\w ]+: \+0)",([\d\.]+),"(\d+)"\]', resp):
        date, price, vol = match.group(1), match.group(2), match.group(3)
        vol = int(vol)
        price = float(price)
        date = " ".join(date.split(" ")[:3])
        df_data.append([date, price, vol])
    df = pd.DataFrame(df_data, columns=["date", "price", "volume"])
    m = re.search(r'"type":"([^"]+?)","market_name"', resp)
    if m is None:
        return None
    mc = re.search(r'"value":"([\w\- ]+ Collection)","color":"9da1a9"', resp)
    if mc is None:
        collection = ""
    else:
        collection = mc.group(1)
    items = [
        (im.group(1), im.group(2), color_to_rare.get(im.group(3), im.group(3)))
        for im in re.finditer(
            r'"value":"([\w\- ]+) \| ([\w\- ]+)","color":"(\w+)"', resp
        )
    ]
    type_ = m.group(1).replace("StatTrak\\u2122", "").replace("\\u2605 ", "").strip()
    data = {"type": type_, "prices": df, "items": items, "collection": collection}
    return data

In [4]:
if os.path.exists("dataset.pkl"):
    with open("dataset.pkl", "rb") as f:
        dataset = pickle.load(f)
else:
    dataset = {}

In [ ]:
header_idx = 0
headers = [
    {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    },
]

base_url = "https://steamcommunity.com/market/search/render/"

window_size = 100
i = 0
while True:
    print("Cursor", i)
    try:
        page = requests.get(
            base_url
            + "?start={}&count={}&search_descriptions=0&appid=730&norender=1".format(
                i, window_size
            ),
            headers=headers[header_idx % len(headers)],
        )
        page_json = page.json()
        assert len(page_json["results"]) > 0
    except (TypeError, OSError, AssertionError) as e:
        print("Load error", e, page.text)
        header_idx += 1
        for _ in range(20):
            time.sleep(3)
        continue
    for res in page_json["results"]:
        weap = parse_weapon(res)
        if weap is None or weap["name"] in dataset:
            continue
        name = weap["name"]
        price_resp = requests.get(
            weap["url"], headers=headers[header_idx % len(headers)]
        ).text
        data = get_prices(price_resp)
        if data is None:
            continue
        print(name, weap["url"])
        weap.update(data)
        dataset[name] = weap.copy()
    i += window_size

Cursor 0
Recoil Case https://steamcommunity.com/market/listings/730/Recoil%20Case
Danger Zone Case https://steamcommunity.com/market/listings/730/Danger%20Zone%20Case
Operation Wildfire Case https://steamcommunity.com/market/listings/730/Operation%20Wildfire%20Case
Horizon Case https://steamcommunity.com/market/listings/730/Horizon%20Case
Operation Phoenix Weapon Case https://steamcommunity.com/market/listings/730/Operation%20Phoenix%20Weapon%20Case
CS20 Case https://steamcommunity.com/market/listings/730/CS20%20Case
Falchion Case https://steamcommunity.com/market/listings/730/Falchion%20Case
Anubis Collection Package https://steamcommunity.com/market/listings/730/Anubis%20Collection%20Package
Clutch Case https://steamcommunity.com/market/listings/730/Clutch%20Case
Chroma Case https://steamcommunity.com/market/listings/730/Chroma%20Case
Shadow Case https://steamcommunity.com/market/listings/730/Shadow%20Case
AWP | Asiimov (Field-Tested) https://steamcommunity.com/market/listings/730/AW

In [15]:
dataset

{'Dreams & Nightmares Case': {'name': 'Dreams & Nightmares Case',
  'stattrak': False,
  'weapon': 'Case',
  'skin': 'n/a',
  'condition': 'n/a',
  'sell_price': 1.15,
  'sell_listings': 232962,
  'url': 'https://steamcommunity.com/market/listings/730/Dreams%20%26%20Nightmares%20Case',
  'is_key': False,
  'is_case': True,
  'is_package': False,
  'type': 'Base Grade Container',
  'prices':              date   price  volume
  0     Jan 21 2022  10.675   76680
  1     Jan 22 2022   6.173   70330
  2     Jan 23 2022   4.914   62767
  3     Jan 24 2022   4.328   52950
  4     Jan 25 2022   3.996   51942
  ...           ...     ...     ...
  1373  Dec 10 2023   1.150    2646
  1374  Dec 10 2023   1.187    2750
  1375  Dec 10 2023   1.181    2495
  1376  Dec 10 2023   1.180    2165
  1377  Dec 10 2023   1.180    2814
  
  [1378 rows x 3 columns],
  'items': [('Five-SeveN', 'Scrawl', 'Mil-Spec'),
   ('MAC-10', 'Ensnared', 'Mil-Spec'),
   ('MAG-7', 'Foresight', 'Mil-Spec'),
   ('P2000', 'Lift

In [17]:
assert len(dataset) > 0
print(len(dataset))
with open("dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)

16


In [20]:
df = pd.DataFrame(dataset)

In [22]:
df.transpose().reset_index(drop=True)

,name,stattrak,weapon,skin,condition,sell_price,sell_listings,url,is_key,is_case,is_package,type,prices,items,collection
0,Dreams & Nightmares Case,False,Case,n/a,n/a,1.15,232962,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Jan 21...,"[(Five-SeveN, Scrawl, Mil-Spec), (MAC-10, Ensn...",
1,Fracture Case,False,Case,n/a,n/a,0.63,282474,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Aug 07...,"[(Negev, Ultralight, Mil-Spec), (P2000, Gnarle...",
2,Revolution Case,False,Case,n/a,n/a,0.75,233455,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Feb 10 2...,"[(MAG-7, Insomnia, Mil-Spec), (MP9, Featherwei...",
3,Prisma 2 Case,False,Case,n/a,n/a,0.93,106794,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Mar 31...,"[(AUG, Tom Cat, Mil-Spec), (AWP, Capillary, Mi...",
4,Chroma 2 Case,False,Case,n/a,n/a,2.7,37671,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Apr 16 ...,"[(AK-47, Elite Build, Mil-Spec), (MP7, Armor C...",
5,Operation Breakout Weapon Case,False,Case,n/a,n/a,6.48,32118,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Jul 02 ...,"[(MP7, Urban Hazard, Mil-Spec), (Negev, Desert...",
6,Chroma 3 Case,False,Case,n/a,n/a,2.43,42992,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Apr 28 ...,"[(Dual Berettas, Ventilators, Mil-Spec), (G3SG...",
7,Spectrum 2 Case,False,Case,n/a,n/a,1.93,69447,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Sep 14...,"[(Sawed-Off, Morris, Mil-Spec), (AUG, Triqua, ...",
8,Gamma Case,False,Case,n/a,n/a,2.44,33522,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 Jun 16...,"[(Five-SeveN, Violent Daimyo, Mil-Spec), (MAC-...",
9,Snakebite Case,False,Case,n/a,n/a,0.32,354231,https://steamcommunity.com/market/listings/730...,False,True,False,Base Grade Container,date price volume 0 May 03...,"[(SG 553, Heavy Metal, Mil-Spec), (Glock-18, C...",
